In [1]:
#include "env_cling.h"

In [2]:
#include <string>
#include <iostream>
#include "xplot/xfigure.hpp"
#include "xplot/xmarks.hpp"
#include "xplot/xaxes.hpp"
#include "xwidgets/xdropdown.hpp"
#include <gsl/gsl_rng.h>
#include <gsl/gsl_randist.h>
#include <gsl/gsl_cdf.h>


#include "phenoFunctions.hxx"
#include "utils.hxx"
#include "jupyterFunctions.hxx"



In [3]:
using S= std::string;
using VS=std::vector<S>;
using VI=std::vector<int>;
using VD=std::vector<double>;
using D=double;
using PVI=std::pair<VI,VI>;

utils::LibUtils U;
using V=vnl_vector<D>;
using T=std::tuple<D,D,D,V>;
auto phFs(pheno::fGaussian<V>);

In [4]:
VD x(1600-600);
int k=600;
std::generate(x.begin(), x.end(), [&k]{ return k+=1; });

In [5]:
size_t n=1600-600;

In [6]:
V data{n};
V w{n};
V y{n};
V w_norm{n};

In [7]:
V t(x.data(),x.size());

In [8]:
gsl_rng * r;
gsl_rng_env_setup();
r = gsl_rng_alloc(gsl_rng_default);

In [9]:
for (size_t i = 0; i < n; i++)
{
    double yi = 50*exp(-std::pow(((t[i]-1300)/400),2));
    y[i]=yi;
    double si = 2;
    double dy = gsl_ran_gaussian(r, 2);
    data[i] = yi + dy;
    w[i] = 1.0 / (si * si);
};


In [10]:
xpl::figure fig;
fig

A Jupyter widget

In [11]:
xpl::linear_scale sx, sy;
xpl::lines line(sx,sy);
line.x=t;
line.y=y;

In [12]:
line.stroke_width=1;
fig.add_mark(line);
xpl::axis hx(sx), hy(sy);
hy.orientation = "vertical";
fig.add_axis(hx);
fig.add_axis(hy);

In [13]:
xpl::lines pts(sx,sy);
pts.x=t;
pts.y=data;

In [14]:
pts.marker="circle";
pts.marker_size=5;
pts.colors=std::vector<std::string>({"black"});
pts.stroke_width=0;
fig.add_mark(pts);

In [15]:
inv::ParameterCostFunction f{3, static_cast<unsigned int>(t.size()),data,t,w,phFs};

In [16]:
inv::VnlLevenbergMarquart L{f};

In [17]:
auto initParameters=pheno::gaussianInitParameters<V,V>(data,t);

In [18]:
L.minimize(initParameters);

In [19]:
auto yHat=phFs(t,initParameters);        

In [20]:
xpl::lines fit(sx,sy);
fit.x=t;
fit.y=yHat;

In [21]:
fit.stroke_width=2;
fit.line_style="dashed";
fit.colors=std::vector<std::string>({"red"});
fig.add_mark(fit);

In [22]:
std::cout<<initParameters<<std::endl;

49.9102 1299.68 401.491


In [23]:
auto residuals=L.getResiduals(yHat);
auto cost=L.getCost(residuals);
auto reducedCost=L.getReducedCost(residuals);
auto prob=L.getCostProba(residuals,0.05);

In [24]:
cost

951.679

In [25]:
reducedCost

0.954542

In [26]:
prob

0.845126

# Normalisation

In [27]:
auto tuple=pheno::normalizedProfile<V,pheno::TupleType>(data,t);
auto prof=std::get<3>(tuple);
auto t_max=std::get<2>(tuple);
auto vmin=std::get<0>(tuple);
auto vmax=std::get<1>(tuple);

In [28]:
xpl::figure fig_norm;
fig_norm

A Jupyter widget

In [29]:
xpl::linear_scale sx_norm, sy_norm;
xpl::lines line_norm(sx_norm,sy_norm);
line_norm.x=t;
line_norm.y=prof;

In [30]:
line_norm.stroke_width=0;
line_norm.marker="circle";
line_norm.marker_size=2;
fig_norm.add_mark(line_norm);

In [31]:
xpl::axis hx_norm(sx_norm), hy_norm(sy_norm);
hy_norm.orientation = "vertical";
fig_norm.add_axis(hx_norm);
fig_norm.add_axis(hy_norm);

In [32]:
for (size_t i = 0; i < n; i++)
{
    double si = 2/(vmax-vmin);
    w_norm[i] = 1.0 / (si*si);
};

In [33]:
auto initParameters_norm=pheno::gaussianInitParameters<V,V>(prof,t);

In [34]:
std::cout<<initParameters_norm<<std::endl;

1 1328 272


In [35]:
inv::ParameterCostFunction fNorm{3, static_cast<unsigned int>(t.size()),prof,t,w_norm,phFs};

In [36]:
inv::VnlLevenbergMarquart Lnorm{fNorm};
Lnorm.minimize(initParameters_norm);

In [37]:
std::cout<<initParameters_norm<<std::endl;

0.904552 1299.5 422.119


In [38]:
auto yHat_norm=phFs(t,initParameters_norm);        

In [39]:
xpl::lines fit_norm(sx_norm,sy_norm);
fit_norm.x=t;
fit_norm.y=yHat_norm;

In [40]:
fit_norm.stroke_width=2;
fit_norm.line_style="dashed";
fit_norm.colors=std::vector<std::string>({"black"});
fig_norm.add_mark(fit_norm);

In [41]:
auto residuals_norm=Lnorm.getResiduals(yHat_norm);
auto cost_nom=Lnorm.getCost(residuals_norm);
auto reducedCost_norm=Lnorm.getReducedCost(residuals_norm);
auto prob_nom=Lnorm.getCostProba(residuals_norm,0.05);

In [42]:
cost_nom

1010.97

In [43]:
reducedCost_norm

1.01401

In [44]:
prob_nom

0.372107

In [45]:
using M= vnl_matrix<double> ;

In [46]:
int nbSeeds=1000;

In [47]:
M seeds(n,nbSeeds);

In [48]:
seeds.fill(0);

In [49]:
for(auto i=0;i<nbSeeds;i++){
    V col{t.size()};
    for (auto j=0;j<t.size();j++){
            double yt = 50*exp(-std::pow(((t[i]-1300)/400),2));
            double dyt = gsl_ran_gaussian(r, 2);
            col[j] = yt + dyt;
    }
    seeds.set_column(i,col);
}

In [50]:
V seed_chisq{n};

In [51]:
for(auto k=0;k<n;k++){
    auto params=pheno::gaussianInitParameters<V,V>(seeds.get_column(k),t); 
    inv::ParameterCostFunction PCF{3, static_cast<unsigned int>(t.size()),seeds.get_column(k),t,w,phFs};
    inv::VnlLevenbergMarquart Leven{PCF};
    Leven.minimize(params);
    auto yt=phFs(t,params);  
    auto res=Leven.getResiduals(yt);
    auto c=Leven.getCost(res);
    seed_chisq[k]=c;
}

In [52]:
auto chisq_mean=seed_chisq.mean()

In [53]:
chisq_mean

999.201

In [54]:
auto deviation=std::sqrt(seed_chisq.squared_magnitude()/n);

In [55]:
deviation

1001.57

In [56]:
xpl::linear_scale xHist, yHist;
xpl::hist hist(xHist, yHist);
hist.sample = seed_chisq;

In [57]:
xpl::figure fig_hist;
fig_hist.add_mark(hist);
fig_hist

A Jupyter widget

In [58]:
xpl::axis hxHist(xHist), hyHist(yHist);
hyHist.orientation = "vertical";
fig_hist.add_axis(hxHist);
fig_hist.add_axis(hyHist);